### Pipeline test ###

In this notebook we will try a possible textual claim generation to be implemented later as part of the package

In [3]:
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize,compose
from src.evidence import FeverousRandomRetriever
from src.pipeline import ClaimGeneratorPipeline
from src.claim import FeverousGenerator
import numpy as np


GlobalHydra.instance().clear()
initialize(config_path='../src/config/', job_name="test_app")

hydra.initialize()

In [4]:
cfg = compose(config_name="config_pipeline")
rng = np.random.default_rng(cfg.seed)

retriever = FeverousRandomRetriever(cfg.data_path,
                                         cfg.num_evidence,
                                         cfg.n_pieces,
                                         cfg.table_per_page,
                                         cfg.evidence_per_table,
                                         cfg.column_per_table,
                                         cfg.seed
                                         )
generator = FeverousGenerator(cfg.model_path)

pipeline = ClaimGeneratorPipeline([retriever,generator])
output,text_evidence = pipeline.generate(None) # Right now, FeverousRetriever doesn't support an input table

#output, text_evidence = generator.generate(output)

for i,(o,e) in enumerate(zip(output,text_evidence)):
    print(f'Claim {i}: ')
    print('Generated: ', o)
    print('Evidence : ', e)


Claim 0: 
Generated:  <pad> Jennifer L. Martin is a doctoral advisor at Oxford University.</s>
Evidence :  Oxford University]] (PhD)
[[London_Business_Scho && Jennifer L. Martin && Alma mater | Louise Johnson && Jennifer L. Martin && Doctoral advisor
Claim 1: 
Generated:  <pad> Serra Miyeun Hwang was born in the United States and has been a
Evidence :  Born && Serra Miyeun Hwang && Born | 황미연 && Serra Miyeun Hwang && Hangul
Claim 2: 
Generated:  <pad> Băsești is a city of the city of the city of the city of the city
Evidence :  Mayor && Băsești && Mayor | Population (2011) && Băsești && Population (2011)
Claim 3: 
Generated:  <pad> Beecher (community), Wisconsin, has a number of Area codes.</s>
Evidence :  Area code(s) && Beecher (community), Wisconsin && Area code(s) | Country && Beecher (community), Wisconsin && Country
Claim 4: 
Generated:  <pad> [[Flora_fami]] is a small planet in the main-
Evidence :  14.0 · 14.1 · 14.54±0.086 && 9260 Edwardolson && Absolute magnitude (H)